In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import KFold
import csv
import seaborn as sns
plt.rc('figure', figsize=[12,8])

/home/alex/miniconda2/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


# Import data, PCA, K-means

In [2]:
traindf = pd.read_csv('train.csv')
testdf = pd.read_csv('test.csv')

traindf.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
testdf.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
xdata = traindf.drop(['label'], axis=1).values
ydata = traindf['label'].values

# pca = PCA(n_components=50)
# pca.fit(xdata)
# print(sum(pca.explained_variance_ratio_))

In [ ]:
X = pca.transform(xdata)
ypred = KMeans(n_clusters=10).fit_predict(X)

In [ ]:
sns.violinplot(ydata, ypred)

Looks like K-means clustering is ineffective (clusters are probably different sizes) - maybe knn will be better.

# KNN

In [ ]:
n_neighbours = range(1, 7, 1)
results = []
for n in n_neighbours:
    scores = []
    for train, test in KFold(n=len(ydata), n_folds=4, shuffle=True):
        neigh = KNeighborsClassifier(n_neighbors=n, n_jobs=-1)
        neigh.fit(X[train], ydata[train])
        scores.append(neigh.score(X[test], ydata[test]))
    results.append([np.mean(scores), min(scores), max(scores)])
    
plt.plot(n_neighbours, [r[0] for r in results])
plt.plot(n_neighbours, results)
plt.fill_between(n_neighbours, [r[1] for r in results], [r[2] for r in results], alpha=0.2)

In [ ]:
scores = []
for train, test in KFold(n=len(ydata), n_folds=4, shuffle=True):
    neigh = KNeighborsClassifier(n_neighbors=4, n_jobs=-1)
    neigh.fit(X[train], ydata[train])
    scores.append(neigh.score(X[test], ydata[test]))
print scores

In [ ]:
sns.violinplot(ydata, neigh.predict(X))

Looks good!

In [ ]:
testdata = testdf.values
XVal = pca.transform(testdata)
predictions = neigh.predict(XVal)

ImageId = range(1, len(predictions)+1)

predictions_file = open("avrknn.csv", "wb")
open_file_object = csv.writer(predictions_file)
open_file_object.writerow(['ImageId', 'Label'])
open_file_object.writerows(zip(ImageId, predictions))
predictions_file.close()

# Neural net

## Pybrain

In [5]:
ydata_net = np.zeros([len(ydata), 10])
for i, y in enumerate(ydata):
    ydata_net[i, y] = 1

In [ ]:
# Try a neural network
import pybrain
from pybrain.tools.shortcuts import buildNetwork
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.datasets import SupervisedDataSet
    
ds = SupervisedDataSet(len(xdata[0]), 10)

for i, j in zip(xdata, ydata_net):
    ds.addSample(i, j)
    
net = buildNetwork(len(xdata[0]), 50, 10, bias=True)
trainer = BackpropTrainer(net, ds)

for i in range(20):
    print trainer.train()

output = [net.activate(d) for d in xdata]
predictions = [i.argmax() for i in output]
matches = (predictions == ydata)
accuracy = float(matches.sum()) / float(len(matches))
print accuracy

In [ ]:
sns.violinplot(ydata, predictions)

Neural net appears to struggle distinguishing some characters.

## Tensorflow